In [63]:
import pandas as pd
import numpy as np
from keras.layers import Dense, Flatten, Embedding, Input, merge, Dropout
from keras.regularizers import l2
from keras.models import Model
from keras.optimizers import Adam

In [4]:
movies = pd.read_csv("ml-latest-small/movies.csv")

In [3]:
ml.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
ratings = pd.read_csv("ml-latest-small/ratings.csv")

In [8]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [9]:
all_users = ratings.userId.unique()
all_movies = ratings.movieId.unique()

In [11]:
all_users[:10]

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [12]:
all_movies[:10]

array([  31, 1029, 1061, 1129, 1172, 1263, 1287, 1293, 1339, 1343])

In [13]:
user_mapping = {o:i for i,o in enumerate(all_users)}

In [14]:
movie_mapping = {o:i for i,o in enumerate(all_movies)}

In [17]:
ratings['user_idx'] = ratings.apply(lambda x: user_mapping[x['userId']], axis=1)
ratings['movie_idx'] = ratings.apply(lambda x: movie_mapping[x['movieId']], axis=1)

In [25]:
trn_idx = np.random.rand(len(ratings)) < .8
trn = ratings[trn_idx]
val = ratings[~trn_idx]

In [26]:
trn.head()

,userId,movieId,rating,timestamp,user_idx,movie_idx
1,1,1029,3.0,1260759179,0,1
3,1,1129,2.0,1260759185,0,3
5,1,1263,2.0,1260759151,0,5
6,1,1287,2.0,1260759187,0,6
7,1,1293,2.0,1260759148,0,7


In [27]:
val.head()

,userId,movieId,rating,timestamp,user_idx,movie_idx
0,1,31,2.5,1260759144,0,0
2,1,1061,3.0,1260759182,0,2
4,1,1172,4.0,1260759205,0,4
8,1,1339,3.5,1260759125,0,8
11,1,1405,1.0,1260759203,0,11


In [34]:
n_users = len(all_users)
n_movies = len(all_movies)
n_factors = 50

In [49]:
np.random.seed = 42

In [58]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users, n_factors, input_length=1, W_regularizer=l2(1e-4))(user_in)

movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-4))(movie_in)

x = merge([u, m], mode='dot')
x = Flatten()(x)

model = Model([user_in, movie_in], x)
model.compile(Adam(0.0001), loss='mse')
model.fit([trn.user_idx, trn.movie_idx], trn.rating, batch_size=64, nb_epoch=100, 
          validation_data=([val.user_idx, val.movie_idx], val.rating))

Train on 80095 samples, validate on 19909 samples
Epoch 1/100
80095/80095 [==============================] - 2s - loss: 13.6872 - val_loss: 13.7060
Epoch 2/100
80095/80095 [==============================] - 2s - loss: 13.6666 - val_loss: 13.6846
Epoch 3/100
80095/80095 [==============================] - 2s - loss: 13.5873 - val_loss: 13.5333
Epoch 4/100
80095/80095 [==============================] - 2s - loss: 13.2441 - val_loss: 12.9845
Epoch 5/100
80095/80095 [==============================] - 2s - loss: 12.4065 - val_loss: 11.9166
Epoch 6/100
80095/80095 [==============================] - 2s - loss: 11.0950 - val_loss: 10.4697
Epoch 7/100
80095/80095 [==============================] - 2s - loss: 9.5133 - val_loss: 8.8789
Epoch 8/100
80095/80095 [==============================] - 2s - loss: 7.9153 - val_loss: 7.3922
Epoch 9/100
80095/80095 [==============================] - 2s - loss: 6.5291 - val_loss: 6.1956
Epoch 10/100
80095/80095 [==============================] - 2s - loss: 5.4

KeyboardInterrupt: 

In [59]:
model.optimizer.lr = .00001
model.fit([trn.user_idx, trn.movie_idx], trn.rating, batch_size=64, nb_epoch=100, 
          validation_data=([val.user_idx, val.movie_idx], val.rating))

Train on 80095 samples, validate on 19909 samples
Epoch 1/100
80095/80095 [==============================] - 2s - loss: 1.9436 - val_loss: 2.5295
Epoch 2/100
80095/80095 [==============================] - 2s - loss: 1.9405 - val_loss: 2.5301
Epoch 3/100
80095/80095 [==============================] - 2s - loss: 1.9375 - val_loss: 2.5307
Epoch 4/100
80095/80095 [==============================] - 2s - loss: 1.9346 - val_loss: 2.5311
Epoch 5/100
80095/80095 [==============================] - 2s - loss: 1.9317 - val_loss: 2.5321
Epoch 6/100
80095/80095 [==============================] - 2s - loss: 1.9291 - val_loss: 2.5329
Epoch 7/100
80095/80095 [==============================] - 2s - loss: 1.9264 - val_loss: 2.5336
Epoch 8/100
80095/80095 [==============================] - 2s - loss: 1.9238 - val_loss: 2.5341
Epoch 9/100
80095/80095 [==============================] - 2s - loss: 1.9213 - val_loss: 2.5348
Epoch 10/100
80095/80095 [==============================] - 2s - loss: 1.9188 - val_lo

KeyboardInterrupt: 

In [45]:
model.compile(Adam(0.0001), loss='mse')
model.fit([trn.user_idx, trn.movie_idx], trn.rating, batch_size=64, nb_epoch=6, 
          validation_data=([val.user_idx, val.movie_idx], val.rating))

Train on 80095 samples, validate on 19909 samples
Epoch 1/6
80095/80095 [==============================] - 2s - loss: 3.3425 - val_loss: 2.9711
Epoch 2/6
80095/80095 [==============================] - 2s - loss: 2.5042 - val_loss: 2.6431
Epoch 3/6
80095/80095 [==============================] - 2s - loss: 2.2221 - val_loss: 2.5596
Epoch 4/6
80095/80095 [==============================] - 2s - loss: 2.1125 - val_loss: 2.5445
Epoch 5/6
80095/80095 [==============================] - 2s - loss: 2.0606 - val_loss: 2.5438
Epoch 6/6
80095/80095 [==============================] - 2s - loss: 2.0317 - val_loss: 2.5519


In [46]:
model.compile(Adam(0.0001), loss='mse')
model.fit([trn.user_idx, trn.movie_idx], trn.rating, batch_size=64, nb_epoch=6, 
          validation_data=([val.user_idx, val.movie_idx], val.rating))

Train on 80095 samples, validate on 19909 samples
Epoch 1/6
80095/80095 [==============================] - 2s - loss: 2.1513 - val_loss: 2.6840
Epoch 2/6
80095/80095 [==============================] - 2s - loss: 2.0871 - val_loss: 2.6580
Epoch 3/6
80095/80095 [==============================] - 2s - loss: 2.0559 - val_loss: 2.6393
Epoch 4/6
80095/80095 [==============================] - 2s - loss: 2.0309 - val_loss: 2.6244
Epoch 5/6
80095/80095 [==============================] - 2s - loss: 2.0101 - val_loss: 2.6128
Epoch 6/6
80095/80095 [==============================] - 2s - loss: 1.9924 - val_loss: 2.6032


# Neural Net Model

In [73]:
user_input = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users, n_factors, input_length=1, W_regularizer=l2(1e-4))(user_input)
movie_input = Input(shape=(1,), dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-4))(movie_input)

In [74]:
x = merge([u, m], mode='concat', concat_axis=1)

In [75]:
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(70, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)
nn = Model([user_input, movie_input], x)
nn.compile(Adam(0.001), loss='mse')
nn.fit([trn.user_idx, trn.movie_idx], 
       trn.rating, 
       batch_size=64, 
       nb_epoch=3, 
       validation_data=([val.user_idx, val.movie_idx], val.rating))

Train on 80095 samples, validate on 19909 samples
Epoch 1/3
80095/80095 [==============================] - 3s - loss: 2.4588 - val_loss: 0.9028
Epoch 2/3
80095/80095 [==============================] - 3s - loss: 1.4620 - val_loss: 0.8797
Epoch 3/3
80095/80095 [==============================] - 3s - loss: 1.2273 - val_loss: 0.8552


In [76]:
nn.fit([trn.user_idx, trn.movie_idx], 
       trn.rating, 
       batch_size=64, 
       nb_epoch=3, 
       validation_data=([val.user_idx, val.movie_idx], val.rating))

Train on 80095 samples, validate on 19909 samples
Epoch 1/3
80095/80095 [==============================] - 3s - loss: 1.0411 - val_loss: 0.8460
Epoch 2/3
80095/80095 [==============================] - 3s - loss: 0.9202 - val_loss: 0.8349
Epoch 3/3
80095/80095 [==============================] - 3s - loss: 0.8478 - val_loss: 0.8262


In [77]:
nn.fit([trn.user_idx, trn.movie_idx], 
       trn.rating, 
       batch_size=64, 
       nb_epoch=10, 
       validation_data=([val.user_idx, val.movie_idx], val.rating))

Train on 80095 samples, validate on 19909 samples
Epoch 1/10
80095/80095 [==============================] - 3s - loss: 0.8213 - val_loss: 0.8266
Epoch 2/10
80095/80095 [==============================] - 3s - loss: 0.8085 - val_loss: 0.8301
Epoch 3/10
80095/80095 [==============================] - 3s - loss: 0.8049 - val_loss: 0.8334
Epoch 4/10
80095/80095 [==============================] - 3s - loss: 0.8003 - val_loss: 0.8244
Epoch 5/10
80095/80095 [==============================] - 3s - loss: 0.7974 - val_loss: 0.8277
Epoch 6/10
80095/80095 [==============================] - 3s - loss: 0.7994 - val_loss: 0.8260
Epoch 7/10
80095/80095 [==============================] - 3s - loss: 0.7954 - val_loss: 0.8283
Epoch 8/10
80095/80095 [==============================] - 3s - loss: 0.7926 - val_loss: 0.8309
Epoch 9/10
80095/80095 [==============================] - 3s - loss: 0.7926 - val_loss: 0.8272
Epoch 10/10
80095/80095 [==============================] - 3s - loss: 0.7899 - val_loss: 0.8292

In [78]:
nn.compile(Adam(0.0001), loss='mse')
nn.fit([trn.user_idx, trn.movie_idx], 
       trn.rating, 
       batch_size=64, 
       nb_epoch=10, 
       validation_data=([val.user_idx, val.movie_idx], val.rating))

Train on 80095 samples, validate on 19909 samples
Epoch 1/10
80095/80095 [==============================] - 3s - loss: 0.7623 - val_loss: 0.8039
Epoch 2/10
80095/80095 [==============================] - 3s - loss: 0.7572 - val_loss: 0.8007
Epoch 3/10
80095/80095 [==============================] - 3s - loss: 0.7476 - val_loss: 0.7989
Epoch 4/10
80095/80095 [==============================] - 3s - loss: 0.7423 - val_loss: 0.7976
Epoch 5/10
80095/80095 [==============================] - 3s - loss: 0.7348 - val_loss: 0.7973
Epoch 6/10
80095/80095 [==============================] - 3s - loss: 0.7284 - val_loss: 0.7967
Epoch 7/10
80095/80095 [==============================] - 3s - loss: 0.7260 - val_loss: 0.7961
Epoch 8/10
80095/80095 [==============================] - 3s - loss: 0.7239 - val_loss: 0.7959
Epoch 9/10
80095/80095 [==============================] - 3s - loss: 0.7200 - val_loss: 0.7958
Epoch 10/10
80095/80095 [==============================] - 3s - loss: 0.7182 - val_loss: 0.7954

In [79]:
nn.compile(Adam(0.00001), loss='mse')
nn.fit([trn.user_idx, trn.movie_idx], 
       trn.rating, 
       batch_size=64, 
       nb_epoch=10, 

       validation_data=([val.user_idx, val.movie_idx], val.rating))

Train on 80095 samples, validate on 19909 samples
Epoch 1/10
80095/80095 [==============================] - 3s - loss: 0.7148 - val_loss: 0.7932
Epoch 2/10
80095/80095 [==============================] - 3s - loss: 0.7162 - val_loss: 0.7932
Epoch 3/10
80095/80095 [==============================] - 3s - loss: 0.7129 - val_loss: 0.7932
Epoch 4/10
80095/80095 [==============================] - 3s - loss: 0.7165 - val_loss: 0.7932
Epoch 5/10
80095/80095 [==============================] - 3s - loss: 0.7162 - val_loss: 0.7932
Epoch 6/10
80095/80095 [==============================] - 3s - loss: 0.7155 - val_loss: 0.7932
Epoch 7/10
80095/80095 [==============================] - 3s - loss: 0.7141 - val_loss: 0.7931
Epoch 8/10
80095/80095 [==============================] - 3s - loss: 0.7132 - val_loss: 0.7931
Epoch 9/10
80095/80095 [==============================] - 3s - loss: 0.7100 - val_loss: 0.7931
Epoch 10/10
80095/80095 [==============================] - 3s - loss: 0.7163 - val_loss: 0.7931

In [80]:
nn.optimizer.lr = .00005
nn.fit([trn.user_idx, trn.movie_idx], 
       trn.rating, 
       batch_size=64, 
       nb_epoch=10, 
       validation_data=([val.user_idx, val.movie_idx], val.rating))

Train on 80095 samples, validate on 19909 samples
Epoch 1/10
80095/80095 [==============================] - 3s - loss: 0.7122 - val_loss: 0.7930
Epoch 2/10
80095/80095 [==============================] - 3s - loss: 0.7131 - val_loss: 0.7929
Epoch 3/10
80095/80095 [==============================] - 3s - loss: 0.7112 - val_loss: 0.7929
Epoch 4/10
80095/80095 [==============================] - 3s - loss: 0.7089 - val_loss: 0.7930
Epoch 5/10
80095/80095 [==============================] - 3s - loss: 0.7077 - val_loss: 0.7930
Epoch 6/10
80095/80095 [==============================] - 3s - loss: 0.7067 - val_loss: 0.7930
Epoch 7/10
80095/80095 [==============================] - 3s - loss: 0.7099 - val_loss: 0.7930
Epoch 8/10
80095/80095 [==============================] - 3s - loss: 0.7126 - val_loss: 0.7930
Epoch 9/10
80095/80095 [==============================] - 3s - loss: 0.7098 - val_loss: 0.7930
Epoch 10/10
80095/80095 [==============================] - 3s - loss: 0.7097 - val_loss: 0.7930

In [81]:
nn.optimizer.lr = .0001
nn.fit([trn.user_idx, trn.movie_idx], 
       trn.rating, 
       batch_size=64, 
       nb_epoch=10, 
       validation_data=([val.user_idx, val.movie_idx], val.rating))

Train on 80095 samples, validate on 19909 samples
Epoch 1/10
80095/80095 [==============================] - 3s - loss: 0.7063 - val_loss: 0.7930
Epoch 2/10
80095/80095 [==============================] - 3s - loss: 0.7065 - val_loss: 0.7930
Epoch 3/10
80095/80095 [==============================] - 3s - loss: 0.7095 - val_loss: 0.7929
Epoch 4/10
80095/80095 [==============================] - 3s - loss: 0.7078 - val_loss: 0.7930
Epoch 5/10
80095/80095 [==============================] - 3s - loss: 0.7046 - val_loss: 0.7929
Epoch 6/10
80095/80095 [==============================] - 3s - loss: 0.7082 - val_loss: 0.7929
Epoch 7/10
80095/80095 [==============================] - 3s - loss: 0.7064 - val_loss: 0.7929
Epoch 8/10
80095/80095 [==============================] - 3s - loss: 0.7057 - val_loss: 0.7929
Epoch 9/10
80095/80095 [==============================] - 3s - loss: 0.7055 - val_loss: 0.7929
Epoch 10/10
80095/80095 [==============================] - 3s - loss: 0.7020 - val_loss: 0.7929